<a href="https://colab.research.google.com/github/krixik-ai/krixik-docs/blob/main/docs/examples/single_module_pipelines/single_vector-db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import sys
import json
import importlib
from pathlib import Path

# preparación de demo - incuye instanciación de secretos, instalación de requerimientos, y definición de rutas
if os.getenv("COLAB_RELEASE_TAG"):
    # si estás usando este notebook en Google Colab, ingresa tus secretos acá
    MY_API_KEY = "TU_API_KEY_VA_AQUI"
    MY_API_URL = "TU_API_URL_VA_AQUI"

    # si estás usando este notebook en Google Colab, instala requerimientos y descarga los subdirectorios requeridos
    # instala el cliente Python de Krixik
    !pip install krixik

    # instala github-clone, que permite clonación fácil de los subdirectorios del repositorio de documentación XXX
    !pip install github-clone

    # clona los conjuntos de datos
    if not Path("data").is_dir():
        !ghclone XXXX #(in english it's https://github.com/krixik-ai/krixik-docs/tree/main/data)
    else:
        print("ya se clonaron los conjuntos de datos de documentación!")

    # define la variable 'data_dir' para tus rutas
    data_dir = "./data/"

    # crea directorio de salidas
    from pathlib import Path

    Path(data_dir + "/salidas").mkdir(parents=True, exist_ok=True)

    # descarga utilidades
    if not Path("utilities").is_dir():
        !ghclone XXXX # (in english it's https://github.com/krixik-ai/krixik-docs/tree/main/utilities)
    else:
        print("ya has clonado las utilidades de documentación!")
else:
    # si estás usando una descarga local de la documentación, define las rutas relativas a la estructura local de la documentación
    # importa utilidades
    sys.path.append("../../../")

    # define la variable 'data_dir' para tus rutas
    data_dir = "../../../data/"

    # si estás usando este notebook localmente desde el repositorio de documentación Krixik, carga tus secretos de un archivo .env ubicado en la base del repositorio de documentación
    from dotenv import load_dotenv

    load_dotenv("../../../.env")

    MY_API_KEY = os.getenv("MY_API_KEY")
    MY_API_URL = os.getenv("MY_API_URL")


# carga 'reset'
reset = importlib.import_module("utilities.reset")
reset_pipeline = reset.reset_pipeline


# importa Krixik e inicializa sesión con tus secretos personales
from krixik import krixik

krixik.init(api_key=MY_API_KEY, api_url=MY_API_URL)

SUCCESS: You are now authenticated.


## *Pipeline* de Módulo Único: `vector-db` (Base de Datos Vectorial)

Este documento presenta una guía de cómo ensamblar y consumir un *pipeline* de módulo único que solo incluye un módulo [`vector-db` (base de datos vectorial)](../../modulos/modulos_de_bases_de_datos/modulo_vector-db_base_de_datos_vectorial.md).

Ten en cuenta que usar este módulo como módulo único no genera un *pipeline* particularmente fácil de usar, dado que por separado debes tener los archivos NPY que procesarás. Sugerimos detallar este [ejemplo de *pipeline*](../ejemplos_pipelines_de_busqueda/multi_busqueda_semantica_basica.md) y este [otro ejemplo de *pipeline*](../ejemplos_pipelines_de_busqueda/multi_busqueda_semantica_sobre_fragmentos.md), que respectivamente toman archivos de entrada TXT y JSON y habilitan [búsqueda semántica](../../sistema/metodos_de_busqueda/metodo_semantic_search_busqueda_semantica.md) (también llamada búsqueda vectorial) sobre ellos.

El documento se divide en las siguientes secciones:

- [Monta tu *Pipeline*](#monta-tu-pipeline)
- [Formato de Entrada Requerido](#formato-de-entrada-requerido)
- [Cómo Usar el Modelo Predeterminado](#como-usar-el-modelo-predeterminado)
- [El Método `semantic_search`](#el-metodo-semantic_search)
- [Consulta Bases de Datos de Salida Localmente](#consulta-bases-de-datos-de-salida-localmente)

### Monta tu *Pipeline*

Primero crea un *pipeline* de módulo único con un módulo [`vector-db` (base de datos vectorial)](../../modulos/modulos_de_bases_de_datos/modulo_vector-db_base_de_datos_vectorial.md).

Usa el método [`create_pipeline`](../../sistema/creacion_de_pipelines/creacion_de_pipelines.md) para esto, incluyendo solamente una referencia de módulo [`vector-db`](../../modulos/modulos_de_bases_de_datos/modulo_vector-db_base_de_datos_vectorial.md) en el argumento `module_chain`.

In [2]:
# crea un pipeline con un solo módulo vector-db
pipeline = krixik.create_pipeline(name="unico_vector-db_1",
                                  module_chain=["vector-db"])

### Formato de Entrada Requerido

El módulo [`vector-db` (base de datos vectorial)](../../modulos/modulos_de_bases_de_datos/modulo_vector-db_base_de_datos_vectorial.md) acepta archivos NPY como entradas. Estos archivos consisten de un solo arreglo NumPy. Cada fila del arreglo es un vector que el módulo [`vector-db`](../../modulos/modulos_de_bases_de_datos/modulo_vector-db_base_de_datos_vectorial.md) luego indexa para búsqueda semántica (también conocida como búsqueda vectorial).

Antes de procesar un archivo de entrada—uno válido para este *pipeline*—examínalo con el siguiente código:

In [3]:
# examina el contenido de un archivo de entrada válido
import numpy as np

np.load(data_dir + "input/vectores.npy")

array([[0, 1],
       [1, 0],
       [1, 1]])

### Como Usar el Modelo Predeterminado

Ahora procesa el archivo usando el modelo [predeterminado](../../modulos/modulos_de_bases_de_datos/modulo_vector-db_base_de_datos_vectorial.md#modelos-disponibles-en-el-modulo-vector-db) del módulo [`vector-db`](../../modulos/modulos_de_bases_de_datos/modulo_vector-db_base_de_datos_vectorial.md): [`faiss`](https://github.com/facebookresearch/faiss). Por lo pronto, este es el único modelo en este módulo.

Dado que este es el modelo predeterminado, no hace falta que especifiques qué modelo quieres usar a través del argumento opcional [`modules`](../../sistema/parametros_y_procesar_archivos_a_traves_de_pipelines/metodo_process_procesar.md#seleccion-de-modelo-por-medio-del-argumento-modules) del método [`process`](../../sistema/parametros_y_procesar_archivos_a_traves_de_pipelines/metodo_process_procesar.md).

In [4]:
# procesa el archivo con el modelo predeterminado
process_output = pipeline.process(
    local_file_path=data_dir + "input/vectores.npy",  # la ruta de archivo inicial en la que yace el archivo de entrada
    local_save_directory=data_dir + "output",  # el directorio local en el que se guardará el archivo de salida
    expire_time=60 * 30,  # data de este proceso se eliminará del sistema Krixik en 30 minutos
    wait_for_process=True,  # espera que el proceso termine antes de devolver control del IDE al usuario
    verbose=False, # no mostrar actualizaciones de proceso al ejecutar el código
)

La salida del proceso se reproduce con el siguiente código. Para aprender más sobre cada componente de esta salida, revisa la documentación del método [`process`](../../sistema/parametros_y_procesar_archivos_a_traves_de_pipelines/metodo_process_procesar.md).

Dado que la salida de este modelo/módulo es un archivo de base de datos [FAISS](https://github.com/facebookresearch/faiss), `process_output` se muestra como "null". Sin embargo, el archivo de salida se ha guardado en la ubicación indicada bajo `process_output_files`. El `file_id` del archivo procesado es el prefijo del nombre del archivo de salida en esta ubicación.

In [5]:
# nítidamente reproduce la salida de este proceso
print(json.dumps(process_output, indent=2))

{
  "status_code": 200,
  "pipeline": "modules-vector-db-docs",
  "request_id": "536c9e0b-41ed-4c41-99dc-11cdabf32ecc",
  "file_id": "63c88fdc-8b62-4f74-af20-c4816ee0bb88",
  "message": "SUCCESS - output fetched for file_id 63c88fdc-8b62-4f74-af20-c4816ee0bb88.Output saved to location(s) listed in process_output_files.",
  "warnings": [],
  "process_output": null,
  "process_output_files": [
    "../../../data/output/63c88fdc-8b62-4f74-af20-c4816ee0bb88.faiss"
  ]
}


### El Metodo `semantic_search`

Cualquier *pipeline* que contiene un módulo [`vector-db`](../../modulos/modulos_de_bases_de_datos/modulo_vector-db_base_de_datos_vectorial.md) precedido de un módulo [`text-embedder`](../../modulos/modulos_ia/modulo_text-embedder_encaje_lexico.md) tiene acceso al método [`semantic_search`](../../sistema/metodos_de_busqueda/metodo_semantic_search_busqueda_semantica.md). Este te permite hacer búsqueda semántica sobre las bases de datos vectoriales que has creado.

Dado que el *pipeline* de módulo único que acabas de crear carece de un módulo [`text-embedder`](../../modulos/modulos_ia/modulo_text-embedder_encaje_lexico.md), el método [`semantic_search`](../../sistema/metodos_de_busqueda/metodo_semantic_search_busqueda_semantica.md) no funcionará con él. Revisa la documentación de [este ejemplo](../../ejemplos/ejemplos_pipelines_de_busqueda/multi_busqueda_semantica_basica.md) de *pipeline* o de [este otro ejemplo](../../ejemplos/ejemplos_pipelines_de_busqueda/multi_busqueda_semantica_sobre_fragmentos.md) de *pipeline*, los cuales cumplen los requerimientos para este método: el primero recibe archivos TXT, y el segundo recibe archivos JSON.

### Consulta Bases de Datos de Salida Localmente

Además de lo que ofrece el método [`semantic_search`](../../sistema/metodos_de_busqueda/metodo_semantic_search_busqueda_semantica.md), puedes  hacer consultas **localmente** sobre las bases de datos vectoriales que has generado y que cuya ubicación está indicada en `process_output_files`.

La siguiente es una función para localmente hacer búsquedas semánticas sobre el archivo de base de datos antes devuelto.

Nota: Para ejecutar el siguiente código tendrás que instalar la librería `FAISS`. Dependiendo de tus especificaciones locales, instala [faiss-cpu](https://pypi.org/project/faiss-cpu/) o [faiss-gpu](https://pypi.org/project/faiss-gpu/).

In [6]:
# asegúrate que has instalado faiss (faiss-cpu or faiss-gpu)
!pip install faiss-cpu
import faiss
import numpy as np
from typing import Tuple


def query_vector_db(query_vector: np.ndarray, k: int, db_file_path: str) -> Tuple[list, list]:
    # subir base de datos vectorial
    faiss_index = faiss.read_index(db_file_path)

    # ejecutar consulta
    similarities, indices = faiss_index.search(query_vector, k)
    distances = 1 - similarities
    return distances, indices


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


Ahora consulta tu base de datos con la función anterior y un pequeño arreglo de prueba. Los resultados están reproducidos tras el siguiente código:

In [7]:
# haz una búsqueda de prueba con la funcion de búsqueda arriba presentada
original_vectors = np.load(data_dir + "input/vectors.npy")
query_vector = np.array([[0, 1]])
distances, indices = query_vector_db(query_vector, 2, process_output["process_output_files"][0])
print(f"vector de entrada de consulta: {query_vector[0]}")
print(f"vector más cercano al original: {original_vectors[indices[0][0]]}")
print(f"distancia del vector de búsqueda a este vector: {distances[0][0]}")
print(f"segundo vector más cercano al original: {original_vectors[indices[0][1]]}")
print(f"distancia del vector de búsqueda a este vector: {distances[0][1]}")

input query vector: [0 1]
closest vector from original: [0 1]
distance from query to this vector: 0.0
second closest vector from original: [1 1]
distance from query to this vector: 0.2928932309150696


In [8]:
# elimina todos los datos procesados pertenecientes a este pipeline
reset_pipeline(pipeline)